In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import sys

sys.path.append("/nas/ckgfs/users/minhpham/workspace/seed")
sys.path.append("/nas/ckgfs/users/minhpham/workspace/BLINK")

In [29]:
from seed.pipeline.pipeline import Pipeline

In [30]:
from seed.pipeline.retriever import Retriever

retriever = Retriever.get("wiki")

/nas/home/minhpham/miniconda3/envs/ml11/lib/python3.8/site-packages/huggingface_hub/file_download.py:560: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-07-16 22:58:30,748 loading file /nas/home/minhpham/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-07-16 22:58:32,936 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [33]:
from seed.pipeline.verifier import Verifier

verifier = Verifier.get("bart")

In [34]:
from seed.extractor import Extractor

extractor = Extractor.get("unifiedqa")

/nas/home/minhpham/miniconda3/envs/ml11/lib/python3.8/site-packages/huggingface_hub/file_download.py:560: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-07-16 23:26:13,204 loading file /nas/home/minhpham/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-07-16 23:26:15,661 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [24]:
pipeline = Pipeline(retriever, verifier, extractor)

/nas/home/minhpham/miniconda3/envs/ml11/lib/python3.8/site-packages/huggingface_hub/file_download.py:560: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-07-16 22:03:56,787 loading file /nas/home/minhpham/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-07-16 22:03:58,721 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [25]:
import pandas as pd
import jsonlines

df = pd.DataFrame(list(jsonlines.open("../data/totto/totto_test.jsonl")))

In [ ]:
import json

for idx, obj in df.iterrows():
    table = pd.DataFrame([obj["table"]])
    print(table)
    print(obj["label"])
    print(obj["counter_fact"])

    label = obj["label"]

    result = pipeline.run(table)

                 Vacator                                  Reason for change  \
0  Charles L. McNary (R)  Died November 22, 1943. Successor was appointe...   

                Successor Date of successor's formal installation  
0  H. Alexander Smith (R)                        December 7, 1944  
True

Charles L . McNary ( R
Alexander Smith


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.18s/it]


Similarity:  Charles Linza McNary (June 12, 1874February 25, 1944) was an American Republican politician from Oregon. 0.7758303284645081
Similarity:  Charles Linza McNary (June 12, 1874February 25, 1944) was an American Republican politician from Oregon. 0.7512107491493225
Similarity:  Charles Linza McNary (June 12, 1874February 25, 1944) was an American Republican politician from Oregon. 0.8218518495559692
Title: Charles L. McNary --- Sentence: 'Charles Linza McNary (June 12, 1874February 25, 1944) was an American Republican politician from Oregon.' --- Label: {}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.09s/it]


Similarity:  In the Senate, McNary helped to pass legislation that led to the construction of Bonneville Dam on the Columbia River, and worked on agricultural and forestry issues. 0.6422708034515381
Similarity:  In the Senate, McNary helped to pass legislation that led to the construction of Bonneville Dam on the Columbia River, and worked on agricultural and forestry issues. 0.5279567837715149
Similarity:  In the Senate, McNary helped to pass legislation that led to the construction of Bonneville Dam on the Columbia River, and worked on agricultural and forestry issues. 0.5941460132598877
Title: Charles L. McNary --- Sentence: 'In the Senate, McNary helped to pass legislation that led to the construction of Bonneville Dam on the Columbia River, and worked on agricultural and forestry issues.' --- Label: {}


...

Similarity:  In the encirclement battle of Kyiv, the city was acclaimed as a "Hero City", because of its fierce resistance. 0.6792740225791931
Similarity:  In the encirclement battle of Kyiv, the city was acclaimed as a "Hero City", because of its fierce resistance. 0.4317108392715454
Similarity:  In the encirclement battle of Kyiv, the city was acclaimed as a "Hero City", because of its fierce resistance. 0.4794619381427765
Title: Ukraine --- Sentence: 'In the encirclement battle of Kyiv, the city was acclaimed as a "Hero City", because of its fierce resistance.' --- Label: {}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.15s/it]


Similarity:  Although the majority of Ukrainians fought in or alongside the Red Army and Soviet resistance, in Western Ukraine an independent Ukrainian Insurgent Army movement arose (UPA, 1942). 0.6657185554504395
Similarity:  Although the majority of Ukrainians fought in or alongside the Red Army and Soviet resistance, in Western Ukraine an independent Ukrainian Insurgent Army movement arose (UPA, 1942). 0.3193671703338623
Similarity:  Although the majority of Ukrainians fought in or alongside the Red Army and Soviet resistance, in Western Ukraine an independent Ukrainian Insurgent Army movement arose (UPA, 1942). 0.4528770446777344
Title: Ukraine --- Sentence: 'Although the majority of Ukrainians fought in or alongside the Red Army and Soviet resistance, in Western Ukraine an independent Ukrainian Insurgent Army movement arose (UPA, 1942).' --- Label: {}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.05it/s]


Similarity:  "Ukraine: A History.". 0.427108496427536
Similarity:  "Ukraine: A History.". 0.2774749994277954
Similarity:  "Ukraine: A History.". 0.2951468825340271
Title: Ukraine --- Sentence: '"Ukraine: A History.".' --- Label: {}
